In [1]:
import os
import csv
import re
import pandas as pd
import numpy as np

In [2]:
# 载入数据,分离数据集与标签
red_dataset = pd.read_csv(r'data/winequality-red.csv', sep='/')
red_dataset = np.array(red_dataset['fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";' \
                               '"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol"' \
                               ';"quality"'].str.split(';', expand=True).astype(float))
red_label = red_dataset[:,11]
red_data = red_dataset[:,:11]

In [3]:
from collections import Counter
# 已知共有6类
classes = dict(Counter(red_label))
print(classes)

# 扩展标签
classname = np.unique(red_label)
print(classname)
multi_red_label = np.zeros([len(red_label),len(classname)])
for i in range(len(classname)):
    multi_red_label[np.where(red_label==classname[i]),i] = 1
print(multi_red_label)

{5.0: 681, 6.0: 638, 7.0: 199, 4.0: 53, 8.0: 18, 3.0: 10}
[3. 4. 5. 6. 7. 8.]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [4]:
# 类平均向量mc
mc = np.zeros([6,len(red_data[0])])
mc_num = np.zeros([6,1])
for i in range(len(red_label)):
    if red_label[i] == 3.0:
        mc[0] += red_data[i]
        mc_num[0] += 1
        continue
    if red_label[i] == 4.0:
        mc[1] += red_data[i]
        mc_num[1] += 1
        continue
    if red_label[i] == 5.0:
        mc[2] += red_data[i]
        mc_num[2] += 1
        continue
    if red_label[i] == 6.0:
        mc[3] += red_data[i]
        mc_num[3] += 1
        continue
    if red_label[i] == 7.0:
        mc[4] += red_data[i]
        mc_num[4] += 1
        continue
    if red_label[i] == 8.0:
        mc[5] += red_data[i]
        mc_num[5] += 1
        continue
for i in range(len(mc_num)):
    mc[i] /= mc_num[i]
print(mc)

[[ 8.36        0.8845      0.171       2.635       0.1225     11.
  24.9         0.997464    3.398       0.57        9.955     ]
 [ 7.77924528  0.69396226  0.17415094  2.69433962  0.09067925 12.26415094
  36.24528302  0.99654245  3.38150943  0.59641509 10.26509434]
 [ 8.16725404  0.57704112  0.24368576  2.52885463  0.09273568 16.98384728
  56.51395007  0.99710363  3.3049486   0.62096916  9.89970631]
 [ 8.34717868  0.49748433  0.27382445  2.47719436  0.08495611 15.71159875
  40.86990596  0.99661506  3.3180721   0.67532915 10.62951933]
 [ 8.87236181  0.4039196   0.37517588  2.72060302  0.07658794 14.04522613
  35.0201005   0.99610427  3.29075377  0.74125628 11.4659129 ]
 [ 8.56666667  0.42333333  0.39111111  2.57777778  0.06844444 13.27777778
  33.44444444  0.99521222  3.26722222  0.76777778 12.09444444]]


In [5]:
# 总平均向量m_red
m_red = np.zeros(len(red_data[0]))
for i in range(len(red_data[0])):
    m_red[i] = red_data[:, i].mean()
print(m_red)

[ 8.31963727  0.52782051  0.27097561  2.5388055   0.08746654 15.87492183
 46.46779237  0.99674668  3.3111132   0.65814884 10.42298311]


In [6]:
# 构建Sw,Sb，ST
Sw = np.zeros([11,11])
for i in range(6):
    for j in range(len(red_label)):
        if red_label[j] == i + 3.0:
            temp = red_data[j] - mc[i]
            Sw += np.outer(temp ,temp.T)
# print(Sw)
print(Sw.shape)
Sb = np.zeros([11,11])
for i in range(6):
    temp = mc[i] - m_red[i]
    Sb += mc_num[i] * np.outer(temp, temp.T)
# print(Sb)
print(Sb.shape)
ST = Sw + Sb

(11, 11)
(11, 11)


In [7]:
# 求解投影方向
w = np.linalg.eig(np.linalg.pinv(Sw)*Sb)[1][0:8]
print(w)

[[-2.74552829e-04  1.28562299e-02 -2.50568540e-01  2.27669489e-02
  -2.09164760e-01  5.99098685e-02 -7.04469466e-02  9.38155507e-01
  -4.54682008e-02 -3.14779259e-02 -3.21851047e-02]
 [-7.92595834e-04  7.36531677e-02  2.38170779e-02  5.35051615e-01
  -1.48959618e-01 -5.05020254e-01  6.54835603e-01  4.06150372e-02
  -3.31841227e-03  1.19866487e-03  4.58008733e-03]
 [-1.09329735e-04  9.85638358e-02 -9.27002507e-03  8.29664621e-01
   6.63070637e-02  2.35901397e-01 -4.87890329e-01 -6.10991220e-02
  -5.71100167e-03 -8.97707898e-04  2.49966023e-03]
 [-3.60816726e-04  1.07744579e-03 -1.49084771e-02 -5.22253623e-03
  -3.68368652e-02  1.81692269e-03 -1.11458633e-02  2.66870830e-02
   3.01697911e-05  1.79201058e-01  9.82567791e-01]
 [-1.85298819e-03 -9.87557587e-01  1.75382971e-02  1.13132944e-01
  -1.73152293e-03 -9.16431602e-02 -5.41276629e-02  1.69483614e-02
   6.70572301e-04  3.33183511e-04  9.19099031e-04]
 [-1.80956741e-06 -7.99657238e-04  7.14413098e-03 -4.46515340e-03
  -3.37232044e-03 -

In [8]:
# LDA降维与可视化
import matplotlib.pyplot as plt
red_data_lda = np.dot(red_data, w.T)
print(red_data_lda)


if len(red_data_lda[0]) == 2:
    color = ['y', 'gold', 'sandybrown', 'peru', 'red', 'darkred']
    alpha = [0.2, 0.2, 0.4, 0.4, 0.6, 1]
    for i, c, a in zip(classname, color, alpha):
        plt.scatter(red_data_lda[np.where(red_label==i), 0], red_data_lda[np.where(red_label==i), 1], c=c, alpha=a)
    plt.show()

[[ -1.24552872  17.83275899 -12.4017843  ...  -3.30963546   1.78018793
    7.37634503]
 [ -2.72149467  32.75921386 -24.59690021 ...  -3.11483288   1.5628238
    7.74457236]
 [ -2.42351976  29.1288192  -20.87516354 ...  -3.15228709   1.59138906
    7.76558197]
 ...
 [ -0.67075801  12.88317354 -10.7663785  ...  -3.34346164   1.65280319
    6.24187241]
 [ -0.75515235  13.83288997 -12.24856274 ...  -3.37774123   1.82916367
    5.83484068]
 [ -1.5229006   20.43856174 -13.28167166 ...  -3.38212346   1.56431641
    5.96229296]]


In [9]:
# 朴素数据归一化
s = np.std(red_data_lda, axis=0)
m_red_norm = np.zeros(len(red_data_lda[0]))
for i in range(len(red_data_lda[0])):
    m_red_norm[i] = red_data_lda[:, i].mean(0)
red_data_lda_norm = (red_data_lda - m_red_norm)/s

if len(red_data_lda_norm[0]) == 2:
    color = ['y', 'gold', 'sandybrown', 'peru', 'red', 'darkred']
    alpha = [0.2, 0.2, 0.4, 0.4, 0.6, 1]
    for i, c, a in zip(classname, color, alpha):
        plt.scatter(red_data_lda_norm[np.where(red_label==i), 0], red_data_lda_norm[np.where(red_label==i), 1], c=c, alpha=a)
    plt.show()

In [10]:
# softmax判断函数
def softmax(w, b, x):
    s = np.exp(np.dot(w, x.reshape(x.shape[0],1)) + b)
    return s/s.sum()

In [11]:
import math
# 梯度计算函数
def gradient(w, b, x, multi_label):
    gradient_w = np.zeros([6, len(x[0])])
    gradient_b = np.zeros([6, 1])
    for i in range(len(x)):
        J = softmax(w, b, x[i])
        for j in range(6):
            gradient_w[j] += x[i] * (multi_label[i,j] - J[j])
            gradient_b[j] += multi_label[i,j] - J[j] 
    return gradient_w/len(x), gradient_b/len(x)

In [12]:
import time
# logistics回归函数
def logistics_classification(w, b, ita, delta, maxiter, data, label):
    iteration = 0
    w_ = w
    b_ = b
    time_start = time.time()
    while(True):
        gradient_w, gradient_b = gradient(w_, b_, data, label)
        w_ += ita * gradient_w
        b_ += ita * gradient_b
#         if iteration%100 == 0:
#             print("iteration: %d ,w = " %iteration)
#             print(w_)
        iteration += 1
        if np.linalg.norm(gradient_w) < delta or iteration >= maxiter:
            break
    time_end = time.time()
    print('%s  %f s' % ("训练耗时为", (time_end - time_start)))
    return w_, b_

In [13]:
# 对原数据进行logistics回归分类
w0 = np.ones([6, len(red_data[0])])
b0 = np.ones([6, 1])
ita = 1e-3
delta = 0.05
maxiter = 1000
w0, b0 = logistics_classification(w0, b0, ita, delta, maxiter, red_data, multi_red_label)
print(w0)
print(b0)

训练耗时为  85.372345 s
[[0.96881706 1.0003299  0.99841089 0.9927277  0.99992333 0.98812962
  0.95352911 0.99635817 0.98842257 0.99723457 0.95711454]
 [0.95585568 1.00369408 0.996335   1.00150086 0.99984141 0.97892488
  1.01598183 0.99672099 0.99120332 0.99613135 0.95942438]
 [1.06831108 1.02543168 0.99029923 0.99969788 1.00289924 0.99338714
  1.03550471 1.01477415 1.0493356  0.99299416 0.98477641]
 [1.0345603  0.99227556 1.00290618 0.99735658 0.99966282 1.02368132
  1.01195796 1.00348853 1.01228992 1.0087482  1.08491293]
 [1.01538783 0.98226572 1.01202149 1.01941252 0.99832471 1.02147878
  0.99827805 0.99405589 0.97719096 1.00695546 1.0534979 ]
 [0.95706804 0.99600306 1.0000272  0.98930447 0.9993485  0.99439825
  0.98474834 0.99460227 0.98155763 0.99793626 0.96027385]]
[[0.99633936]
 [0.99671342]
 [1.014722  ]
 [1.00352536]
 [0.99410129]
 [0.99459857]]


In [14]:
# 5重交叉验证
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

In [15]:
# 对降维后的数据进行logistic回归分类
# 使用交叉验证，记录每次学习得到的w和b

w_stack = []
b_stack = []
acc_stack = []
ford = 1
for X_train_i,X_test_i in kf.split(red_data_lda_norm):
    w = np.ones([6, len(red_data_lda[0])])
    b = np.ones([6, 1])
    ita = 0.5
    delta = 1e-3
    maxiter = 1000
    w, b = logistics_classification(w, b, ita, delta, maxiter, red_data_lda_norm[X_train_i], multi_red_label[X_train_i])
    
    acc = 0
    predict = np.zeros(len(X_test_i))
    for i in range(len(X_test_i)):
        predict[i] = np.argmax(softmax(w, b, red_data_lda_norm[X_test_i][i])) + 3
    acc = np.sum(predict==red_label[X_test_i])/len(X_test_i)
    print("\033[1;31;47m   Ford%d:acc = %f \033[0m \n" %(ford, acc))
    w_stack.append(w)
    b_stack.append(b)
    acc_stack.append(acc)
    ford += 1

训练耗时为  103.753472 s
   Ford1:acc = 0.553125  

训练耗时为  104.285598 s
   Ford2:acc = 0.534375  

训练耗时为  103.584154 s
   Ford3:acc = 0.584375  

训练耗时为  103.419547 s
   Ford4:acc = 0.534375  

训练耗时为  103.364528 s
   Ford5:acc = 0.608150  



In [16]:
print(acc_stack)

[0.553125, 0.534375, 0.584375, 0.534375, 0.6081504702194357]


In [17]:
def sigmoid(a):
    return 1 / (1 + np.exp(-a))

In [18]:
# 观察分类器在测试集上的表现

In [19]:
# 降维前
predict0 = np.zeros(len(red_label))
for i in range(len(red_label)):
    predict0[i] = np.argmax(softmax(w0, b0, red_data[i])) + 3
acc0 = np.sum(predict0==red_label)/len(red_label)
print("降维前总精度 = %f" %acc0)

降维前总精度 = 0.509694


In [20]:
# 记录分类结果
import csv
with open("redwine_prediction.csv","w", newline="") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["index","label"])
    for i in range(len(predict0)):
        writer.writerow([i,int(predict0[i])])

In [21]:
# 降维后
from numpy import *
w = np.array(w_stack).mean(0)
b = np.array(b_stack).mean(0)
predict = np.zeros(len(red_label))
for i in range(len(red_label)):
    predict[i] = np.argmax(softmax(w, b, red_data_lda_norm[i])) + 3
acc = np.sum(predict==red_label)/len(red_label)
print("降维后总精度 = %f" %acc)

降维后总精度 = 0.580363


In [22]:
# 记录分类结果
import csv
with open("redwine_lda_prediction.csv","w", newline="") as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["index","label"])
    for i in range(len(predict)):
        writer.writerow([i,int(predict[i])])

In [23]:
# 逐类分类
count = np.zeros(6)
acc = np.zeros(6)
for i in range(len(predict)):
    if red_label[i] == 3.0:
        count[0] += 1
        if predict[i] == red_label[i]:
            acc[0] += 1
    if red_label[i] == 4.0:
        count[1] += 1
        if predict[i] == red_label[i]:
            acc[1] += 1
    if red_label[i] == 5.0:
        count[2] += 1
        if predict[i] == red_label[i]:
            acc[2] += 1
    if red_label[i] == 6.0:
        count[3] += 1
        if predict[i] == red_label[i]:
            acc[3] += 1
    if red_label[i] == 7.0:
        count[4] += 1
        if predict[i] == red_label[i]:
            acc[4] += 1
    if red_label[i] == 8.0:
        count[5] += 1
        if predict[i] == red_label[i]:
            acc[5] += 1
print(count)
print(acc)
print(acc/count)

[ 10.  53. 681. 638. 199.  18.]
[  0.   0. 502. 386.  40.   0.]
[0.         0.         0.73715125 0.60501567 0.20100503 0.        ]


In [24]:
# 统计某一类的具体精度
count = 0
acc = 0
for i in range(len(predict)):
    if red_label[i] == 8.0:
        count += 1
        if predict[i] == red_label[i]:
            acc += 1
print(count)
print(acc/count)

18
0.0


In [25]:
# 逐分类器分类
for k in range(6):
    predicts = sigmoid(np.dot(red_data_lda_norm, w[k]) + b[k])
    theta = 0.5
    predict = [1 if i > theta else 0 for i in predicts]
    print(np.sum(predict == multi_red_label[:, k])/len(predict))

0.5797373358348968
0.383364602876798
0.41776110068792993
0.474671669793621
0.40212632895559725
0.6235146966854284


In [26]:
# 单分类器测试
k = 5 # k = 评分
k -= 3
predicts = sigmoid(np.dot(red_data_lda, w[k]) + b[k])
print(predicts)
theta = 0.5
predict = [1 if i > theta else 0 for i in predicts]
print(np.sum(predict == multi_red_label[:, k])/len(predict))

[0.99284139 0.95953843 0.99468368 ... 0.27632768 0.10903502 0.97629388]
0.4446529080675422
